## 14.11 경고 메시지 생성

### 문제

- 프로그램에서 경고 메시지를 표시하고 싶다(더 이상 지원하지 않는 기능이나 사용시 문제점 등)

### 해결

- 프로그램에서 경고 메시지를 생성하려면 warning.warn() 함수를 사용함

In [17]:
%cd 14

/Users/re4lfl0w/Documents/ipython/books/python_cookbook/14


In [4]:
%%writefile warn.py
import warnings

def func(x, y, logfile=None, debug=False):
    if logfile is not None:
        warnings.warn('logfile argument deprecated', DeprecationWarning)
        
func(3, 4, '1.txt')

Writing warn.py


- warn() 경고 메시지와 경고 클래스를 함께 인자로 전달함
- 일반적으로 사용하는 클래스는 UserWarning, DeprecationWarning, SyntaxWarning, RuntimeWarning, ResourceWarning, FutureWarning 등
- 경고 처리는 인터프리터를 어떻게 실행했는지와 환경 설정에 따라 달라짐
- 예를 들어 파이썬을 실행할 때 -W all 옵션을 사용했다면 다음과 같은 결과가 나옴

In [10]:
!python3 warn.py

In [11]:
!python3 -W all warn.py

warn.py:5: DeprecationWarning: logfile argument deprecated
  warnings.warn('logfile argument deprecated', DeprecationWarning)


In [12]:
!python3 -W error warn.py

Traceback (most recent call last):
  File "warn.py", line 7, in <module>
    func(3, 4, '1.txt')
  File "warn.py", line 5, in func
    warnings.warn('logfile argument deprecated', DeprecationWarning)


### 토론

- 경고 메시지 생성하기는 소프트웨어를 유지하고 예외를 발생시키지도 않는, 사소한 문제를 사용자에게 전달할 때 유용한 기술
- 예를 들어 라이브러리나 프레임워크의 동작을 변경할 예정이라면, 기존 기능을 제공하는 동안 앞으로 바뀔 내용을 경고 메시지로 표시할 수 있음
- 그리고 사용자가 코드를 잘못 사용하고 있는 경우에도 경고할 수 있음
- 내장된 라이브러리에서 경고 메시지를 사용하는 예제를 보자
- 다음은 파일을 닫지 않고 제거할 때 발생하는 경고 메시지

In [13]:
import warnings
warnings.simplefilter('always')
f = open('/etc/passwd')
del f

-c:4: ResourceWarning: unclosed file <_io.TextIOWrapper name='/etc/passwd' mode='r' encoding='UTF-8'>


- 기본적으로 모든 경고 메시지가 나타나지는 않음
- 파이썬에 -W 옵션으로 경고 메시지 출력 제어 가능
  - -W all: 모든 경고 메시지 출력
  - -W ignore: 모든 경고 메시지를 무시
  - -W error: 모든 경고를 예외로 변경
- 혹은 warnings.simplefilter() 함수로 결과 출력을 제어 가능
  - always: 모든 경고 메시지가 나타남
  - ignore: 모든 경고를 무시
  - error: 모든 경고를 예외로 변경
- 간단한 경우에 앞에 나온 내용만으로 경고 메시지를 다루는 데 부족함이 없음
- warnings 모듈에 경고 메시지를 걸러 내거나 처리하는 고급 기능이 더 많이 있음

#### warnings 참고 사이트

- [warnings – Non-fatal alerts - Python Module of the Week](http://pymotw.com/2/warnings/)
- [27.6. warnings â Warning control — Python 2.7.8 documentation](https://docs.python.org/2/library/warnings.html)

## 14.12 프로그램 크래시 디버깅

### 문제

- 프로그램에 문제가 생겨서 간단한 방법으로 디버깅하고 싶음

### 해결

- 예외를 발생시키며 프로그램이 크래시했다면, python3 -i someprogram.py 형식으로 프로그램을 실행해서 좀 더 많은 정보를 얻을 수 있음
- -i 옵션을 붙이면 프로그램이 종료하는 즉시 인터랙티브 쉘 시작
- 거기서부터 환경 조사 가능

In [15]:
%%writefile sample.py
def func(n):
    return n + 10

func('Hello')

Writing sample.py


In [16]:
!python3 -i sample.py

Traceback (most recent call last):
  File "sample.py", line 4, in <module>
    func('Hello')
  File "sample.py", line 2, in func
    return n + 10
TypeError: Can't convert 'int' object to str implicitly
>>> 
KeyboardInterrupt

>>> 
>>> 

- 그래도 어디서 잘못된 것인지 확실히 모르겠다면, 크래시 이후 파이썬 debugger를 실행해야 함

In [18]:
def func(n):
    return n + 10

func('Hello')

TypeError: Can't convert 'int' object to str implicitly

In [19]:
import pdb
pdb.pm()

> <ipython-input-18-58e3830df208>(2)func()
-> return n + 10
(Pdb) l
  1  	def func(n):
  2  ->	    return n + 10
  3  	
  4  	func('Hello')
[EOF]
(Pdb) w
  /Users/re4lfl0w/.virtualenvs/python3/lib/python3.4/site-packages/IPython/core/interactiveshell.py(2883)run_code()
-> exec(code_obj, self.user_global_ns, self.user_ns)
  <ipython-input-18-58e3830df208>(4)<module>()
-> func('Hello')
> <ipython-input-18-58e3830df208>(2)func()
-> return n + 10
(Pdb) print n
*** SyntaxError: Missing parentheses in call to 'print'
(Pdb) p n
'Hello'
(Pdb) print(n)
Hello
(Pdb) p n
'Hello'
(Pdb) l
[EOF]
(Pdb) q


In [48]:
%%writefile traceback_test.py
import traceback
import sys

def func(n):
    return n + 10

try:
    func(arg)
except:
    print('**** AN ERROR OCCURRED ****')
    traceback.print_exc(file=sys.stderr)

Writing traceback_test.py


In [49]:
!python3 traceback_test.py

**** AN ERROR OCCURRED ****
Traceback (most recent call last):
  File "/Users/re4lfl0w/Documents/ipython/books/python_cookbook/14/traceback.py", line 8, in <module>
    func(arg)
NameError: name 'arg' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "traceback_test.py", line 1, in <module>
    import traceback
  File "/Users/re4lfl0w/Documents/ipython/books/python_cookbook/14/traceback.py", line 11, in <module>
    traceback.print_exc(file=sys.stderr)
AttributeError: 'module' object has no attribute 'print_exc'


- 프로그램이 크래시하지는 않지만, 예상한 답이 아닌 이상한 값을 생성한다면 여기 저기 print() 호출을 추가해서 살펴볼 수 있다.
- 하지만 이 방식을 사용할 것이라면 관련된 흥미로운 기술이 몇 가지 존재
- 첫째로, traceback.print_stack() 함수는 실행한 지점으로부터 프로그램 스택트레이스를 생성함

In [57]:
%%writefile traceback_print_stack.py
import traceback
import sys

def sample(n):
    if n > 0:
        sample(n-1)
    else:
        traceback.print_stack(file=sys.stderr)
        
sample(5)

Overwriting traceback_print_stack.py


In [58]:
!python3 traceback_print_stack.py

  File "traceback_print_stack.py", line 10, in <module>
    sample(5)
  File "traceback_print_stack.py", line 6, in sample
    sample(n-1)
  File "traceback_print_stack.py", line 6, in sample
    sample(n-1)
  File "traceback_print_stack.py", line 6, in sample
    sample(n-1)
  File "traceback_print_stack.py", line 6, in sample
    sample(n-1)
  File "traceback_print_stack.py", line 6, in sample
    sample(n-1)
  File "traceback_print_stack.py", line 8, in sample
    traceback.print_stack(file=sys.stderr)


- 혹은 pdb.set_trace()를 사용하면 프로그램의 어디서나 디버거를 실행할 수 있음

In [62]:
import pdb

def func(arg):
    pdb.set_trace()
    a = arg * 3
    s = arg + 3    
    
func('argument')

> <ipython-input-62-2e9075848a03>(5)func()
-> a = arg * 3
(Pdb) n
> <ipython-input-62-2e9075848a03>(6)func()
-> s = arg + 3
(Pdb) l
  1  	import pdb
  2  	
  3  	def func(arg):
  4  	    pdb.set_trace()
  5  	    a = arg * 3
  6  ->	    s = arg + 3
  7  	
  8  	func('argument')
[EOF]
(Pdb) p a
'argumentargumentargument'
(Pdb) n
TypeError: Can't convert 'int' object to str implicitly
> <ipython-input-62-2e9075848a03>(6)func()
-> s = arg + 3
(Pdb) c


TypeError: Can't convert 'int' object to str implicitly

- 이 기술은 거대한 프로그램의 실행 흐름이나 함수의 인자와 같이 내부를 살펴볼 때 유용
- 예를 들어 디버거를 시작하고 나면 print로 변수의 값을 보거나, w와 같은 명령어로 스택 트레이스백을 얻을 수 있음

### 토론

- 필요 이상으로 디버깅을 복잡하게 만들 필요 없음
- 간단한 에러는 프로그램 트레이스백을 **읽기만 해도** 해결 가능한 경우가 많음(예: 실제 에러는 대게 트레이스백 마지막 줄에 있음)
- 개발중에는 코드 중간 중간에 print() 함수를 삽입하는 것만으로도 많은 정보를 얻을 수 있음(단, 개발을 마친 후에 잊지 말고 제거해야 함)
- 디버기는 일반적으로 프로그램이 크래시했을 때 함수 내부의 변수 내용을 보는 용도로 사용함
- **크래시가 발생한 후**에 **디버거를 시작하는 방법은 유용**한 기술
- pdb.set_trace() 구문을 넣으면 내부의 흐름이 어떻게 돌아가는지 정확하게 알지 못하는 매우 복잡한 프로그램을 디버깅할 때 유용함
- 근본적으로, 프로그램은 set_trace() 호출이 나올 때까지 실행되고, 호출하는 순간 즉시 디버거로 돌입함. 거기서부터 디버깅을 시작하면 됨
- 파이썬 개발에 IDE를 사용한다면, IDE가 pdb를 사용해서 자체적으로 디버깅 인터페이스를 제공하는 것이 일반적
- 자세한 정보는 사용중인 IDE의 메뉴얼 참고

#### PyCharm debugging

- [Running and Debugging](https://www.jetbrains.com/pycharm/webhelp/running-and-debugging.html)
- [Debugging](https://www.jetbrains.com/pycharm/webhelp/debugging.html)

## 14.13 프로파일링과 타이밍

### 문제

- 프로그램에서 시간을 많이 소비하는 곳이 어디인지 찾고 그 시간을 측정하고 싶다

### 해결

- 프로그램의 전체 실행 시간을 측정하고 싶다면 Unix의 time 명령만으로도 충분하다

In [76]:
%%writefile time_test.py
def add(a, b):
    return a + b

def main(MAX):
    total = 0
    for i in range(MAX):
        total += add(i, i)
        
main(10000000)

Overwriting time_test.py


In [77]:
!time python3 time_test.py


real	0m2.105s
user	0m2.093s
sys	0m0.008s


- 하지만 프로그램이 어떤 일을 하고 있는지 세세한 정보가 필요하다면 cProfile 모듈을 사용해야 함

In [78]:
!python3 -m cProfile time_test.py

         10000004 function calls in 3.259 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    3.259    3.259 time_test.py:1(<module>)
 10000000    0.982    0.000    0.982    0.000 time_test.py:1(add)
        1    2.277    2.277    3.259    3.259 time_test.py:4(main)
        1    0.000    0.000    3.259    3.259 {built-in method exec}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [80]:
import time
from functools import wraps

def timethis(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        r = func(*args, **kwargs)
        end = time.perf_counter()
        print('{}.{} : {}'.format(func.__module__, 
                                  func.__name__, 
                                  end-start))
        return r
    return wrapper

In [82]:
@timethis
def countdown(n):
    while n > 0:
        n -= 1
        
countdown(10000000)

__main__.countdown : 0.7492873570008669


- 구문의 블록 시간을 측정하려면 콘택스트 매니저를 정의함

In [83]:
from contextlib import contextmanager

@contextmanager
def timeblock(label):
    start = time.perf_counter()
    try:
        yield
    finally:
        end = time.perf_counter()
        print('{} : {}'.format(label, end-start))

- 다음은 이 콘택스트 매니저가 동작하는 예제

In [84]:
with timeblock('counting'):
    n = 10000000
    while n > 0:
        n -= 1

counting : 1.3143891820000135


- 특정 코드 조각의 성능을 측정할 때는 timeit 모듈이 유용

In [9]:
from timeit import timeit
timeit('math.sqrt(2)', 'import math')

0.12336524500278756

In [10]:
timeit('sqrt(2)', 'from math import sqrt')

0.08896779199858429

In [11]:
import math
def sqrt1():
    return math.sqrt(2)

In [12]:
%timeit sqrt1()

1000000 loops, best of 3: 205 ns per loop


In [13]:
from math import sqrt
def sqrt2():
    return sqrt(2)

In [14]:
%timeit sqrt2()

10000000 loops, best of 3: 157 ns per loop


- timeit은 첫 번째 인자로 명시된 코드를 백만 번 실행하고 그 시간을 측정함
- 두번째 인자는 테스트를 실행하기 위해 필요한 환경을 설정하는 문자열
- 실행하는 횟수를 조절하고 싶으면 다음과 같이 number 인자를 추가함

In [16]:
timeit('math.sqrt(2)', 'import math', number=10000000)

1.1827796819998184

In [18]:
timeit('sqrt(2)', 'from math import sqrt', number=10000000)

0.8908851390006021

- 성능을 측정할 때, 결과값은 **항상 근사치**라는 점을 유념
- 앞에 나온 에제에서는 time.perf_counter() 함수로 좀 더 정확한 시간을 측정했음
- 하지만 이 역시 벽시계 시간(wall-clock time)을 측정하는 것이고, 컴퓨터의 부하와 같이 다른 요소에 영향을 받을 수 있음
- 벽시계 시간이 아닌 프로세스 시간에 관심이 있다면 time.process_time()을 사용

In [21]:
import time
from functools import wraps

def timethis(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.process_time()
        r = func(*args, **kwargs)
        end = time.process_time()
        print('{}.{} : {}'.format(func.__module__, 
                                  func.__name__, 
                                  end-start))
        return r
    return wrapper

In [22]:
@timethis
def countdown(n):
    while n > 0:
        n -= 1
        
countdown(10000000)

__main__.countdown : 0.7182929999999992


- 마지막으로, 자세한 타이밍 분석을 해야 한다면 time, timeit 등 관련 모듈의 문서를 잘 읽어서 플랫폼의 차이점 등을 잘 이해하도록 함
- 스톱워치 타이머 클래스를 만들어 보았던 레시피 13.13도 참고하도록 한다

#### timeit

- [26.6. timeit â Measure execution time of small code snippets — Python 2.7.8 documentation](https://docs.python.org/2/library/timeit.html)
- [timeit – Time the execution of small bits of Python code. - Python Module of the Week](http://pymotw.com/2/timeit/)

## 14.14 프로그램 실행 속도 향상

### 문제

- 프로그램 실행 속도가 너무 느리다.
- 이때 C 확장이나 just-in-time(JIT) 컴파일러와 같은 극단적인 해결책의 도움없이 속도를 향상시키고 싶다

### 해결

- 최적화의 첫 번째 규칙은 **"하지 않는 것"**이고, 두 번째 규칙은 **"중요하지 않은 것을 최적화 하지 마라"**
- 이 목적을 달성하려면 우선 레시피 14.13에 나왔던 방법으로 코드를 **프로파일링**해야 함
- 대개 프로그램의 실행 시간을 대부분 차지하는 곳은 **내부 데이터 처리 루프** 등 **몇 군데**로 압축됨
- 이 위치를 알아냈다면, 다음에 나오는 기술을 사용해서 프로그램 실행 속도를 향상시켜 보자

#### 함수 사용

- 많은 프로그래머들은 파이썬을 단순한 스크립트 작성에 사용함
- 스크립트를 작성할 때는 아주 간단한 구조로 코드를 쓰는 경우가 많음

```python
# somescript.py
import sys
import csv

with open(sys.argv[1]) as f:
    for row in csv.reader(f):
        # 작업 수행
        ...
```

- 이와 같이 전역 영역에 정의된 코드는 함수에 정의한 코드보다 실행 속도가 느리다는 사실을 아는 사람이 많지 않음
- 속도 차이는 지역과 전역변수의 구현에서 비롯됨(지역 관련 변수의 실행 속도가 더 빠름)
- 따라서 프로그램 실행 속도를 향상시키려면 간단히 스크립트 구문을 함수 안에 넣으면 됨

```python
# somescript.py
import sys
import csv

def main(filename):
    with open(filename) as f:
        for row in csv.reader(f):
            # 작업 수행
            ...
main(sys.argv[1])
```

- 속도 차이는 수행하는 작업에 따라 크게 다르지만, 경험에 따르면 15~30% 정도의 향상이 놀라울 정도는 아니다.

#### 속성 접근의 선택적 삭제

- 점(.) 연산자로 속성에 접근하는 작업에는 대가가 따른다.
- 코드 내부적으로는 \_\_getattribute\_\_()와 \_\_getattr\_\_() 같은 특별 메소드를 호출하느넫, 결국 딕셔너리 검색으로 이어지는 경우가 많음
- 속성 사용은 from module import name 형식의 임포트와 선택적인 바운드 메소드 사용으로 피해갈 수 있음
- 이해를 돕기 위해 다음과 같은 코드가 있다고 가정해 보자

In [28]:
import math
from math import sqrt

def compute_roots1(nums):
    result = []
    for n in nums:
        result.append(math.sqrt(n))
    return result

def compute_roots2(nums):
    result = []
    result_append = result.append
    for n in nums:
        result_append(sqrt(n))
    return result


def main1():
    # 테스트
    nums = range(1000000)
    for i in range(100):
        r = compute_roots1(nums)

def main2():
    # 테스트
    nums = range(1000000)
    for i in range(100):
        r = compute_roots2(nums)

In [30]:
%time main1()

CPU times: user 20.4 s, sys: 927 ms, total: 21.3 s
Wall time: 21.3 s


In [29]:
%time main2()

CPU times: user 14 s, sys: 942 ms, total: 14.9 s
Wall time: 14.9 s


#### 변수의 지역성 이해

- 앞에서 설명한 대로 지역변수는 전역변수보다 실행 속도가 빠름
- 빈번하게 접근하는 이름의 경우 최대한 지역에 가깝게 만들어서 속도 향상을 도모할 수 있음
- 예를 들어 앞에 나왔던 compute_roots() 함수를 다음처럼 수정해 보자

In [31]:
import math

def compute_roots3(nums):
    sqrt = math.sqrt
    result = []
    result_append = result.append
    for n in nums:
        result_append(sqrt(n))
    return result

def main3():
    # 테스트
    nums = range(1000000)
    for i in range(100):
        r = compute_roots3(nums)

In [32]:
%time main3()

CPU times: user 13 s, sys: 898 ms, total: 13.9 s
Wall time: 13.9 s


- 13.9s: 지역변수에서 불러오기
- 14.9s: .연산자 선택적 삭제
- 21.3s: 일반적 접근

- 이 버전에서 sqrt를 math 모듈에서 가져와 지역변수에 넣었다. 이 코드를 실행해 보면 이제 25초로 시간이 다시 줄어들었음(29초에서 4초 향상)
- 이는 sqrt를 전역 레벨에서 찾지않고 **지역 레벨에서 찾는 것이 더 빠르기** 때문에 일어난 현상


- 클래스에서 작업할 때 지역 인자도 적용됨
- 일반적으로 self.name과 같이 값을 찾는 것이 지역변수에 접근하는 것보다 느리다.
- 순환문 내부에서 자주 접근하는 속성을 지역변수에 넣으면 실행 속도가 빨라짐

```python
# 느림
class SomeClass:
    ...
    def method(self):
        for x in s:
            op(self.value)
            
# 빠름
class SomeClass:
    ...
    def method(self):
        value = self.value
        for x in s:
            op(value)
```

#### 불필요한 추상화 피하기

- 데코레이터나 프로퍼티, 디스크립터 등 코드를 추가적인 처리 레이어로 감쌀 때마다 실행 속도가 느려짐

In [33]:
class A:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    @property
    def y(self):
        return self._y
    
    @y.setter
    def y(self, value):
        self._y = value

In [34]:
a = A(1, 2)

In [41]:
a.x

1

In [42]:
a.y

2

In [44]:
a.y = 5

In [45]:
a.y

5

In [35]:
%timeit a.x

10000000 loops, best of 3: 56.1 ns per loop


In [36]:
%timeit a.y

10000000 loops, best of 3: 170 ns per loop


- 이제 간단한 시간 측정

In [37]:
from timeit import timeit

In [38]:
a = A(1, 2)

In [39]:
timeit('a.x', 'from __main__ import a')

0.04903155400097603

In [40]:
timeit('a.y', 'from __main__ import a')

0.18678854100289755

- 보는 것처럼 프로퍼티 y 접근은 단순히 x에 접근하는 것보다 4.5배 이상 느리다.
- 이런 차이점이 문제가 된다면 y 정의에 사용한 프로퍼티가 정말로 필요한 것인지 진지하게 고민해 보아야 함
- 만약 필요 없다고 판단한다면 프로퍼티를 제거하고 단순히 속성을 사용하는 방법을 채택
- 다른 프로그램 언어에서 게터/세터 함수를 사용하는 것이 일반적이라고 해서 파이썬에서 이 방식을 꼭 따라야 할 의무는 없음

#### 내장 컨테이너 사용

- 문자열, 튜플, 리스트, 세트, 딕셔너리 등 내장된 데이터 타입은 모두 C로 구현되어 있고 따라서 실행 속도가 상대적으로 빠름
- 만약 이런 것 대신에 스스로 자료 구조를 만들고 싶다면(예: 링크드 리스트, 밸런스 트리 등), 내장된 자료 구조의 속도를 따라잡기는 거의 불가능
- 따라서 괜히 고생하지 마록 이미 제공되고 있는 것을 사용하도록 하자

#### 불필요한 자료 구조 생성이나 복사 피하기

- 프로그래머들은 종졸 불필요한 자료 구조를 만들기도 함
- 예를 들어 다음 코드를 보자

In [48]:
sequence = range(10)

In [50]:
values = [x for x in sequence]
values

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [51]:
squares = [x*x for x in values]
squares

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

- 아마도 여러 값을 리스트에 넣고 그 값으로 리스트 컴프리헨션과 같은 작업을 하려는 의도일 것
- 하지만 첫 번째 나오는 리스트는 전혀 필요가 없다.
- 앞에 나온 코드는 다음과 같이 개선할 수 있음

In [52]:
squares = [x*x for x in sequence]
squares

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

- 이와 관련해서, 파이썬의 값 공유에 대해 편집증적인 반응을 보이는 프로그래머가 작성한 코드를 주의하도록 함
- copy.deepcopy()와 같은 함수를 남용했다면, 이 코드를 작성한 사람은 파이썬의 메모리 모델을 믿지 못하거나 잘 이해하지 못했을 확률이 크다.
- 그런 코드에서는, 여러 복사본을 제거해도 안전할 것

### 토론

- **최적화를 하기 전**에, 우선적으로 사용하고 있는 **알고리즘에 대한 연구를 하는 것이 좋음**
- O(n\*\*2) 알고리즘을 사용한 코드를 아무리 최적화한다고 해도 O(n log n) 알고리즘을 사용한 코드보다 실행 속도가 훨씬 느릴 것
- 여전히 최적화를 해야 한다고 생각하면, 우선 큰 그림부터 그려 보도록 하자
- 일반적으로 프로그램의 **모든 부분에 최적화를 적용**하면 코드를 **읽기도 어렵고 이해하기도 어렵게** 만든다.
- 세세한 최적화 결과를 해설할 때는 주의해야 함
- 예를 들어 딕셔너리를 생성하는 다음 두 코드를 살펴보자

In [53]:
a = {
'name': 'AAPL',
'shares': 100,
'price': 534.22
}

In [54]:
b = dict(name='AAPL', shares=100, price=534.22)

In [55]:
%timeit a

10000000 loops, best of 3: 25.9 ns per loop


In [56]:
%timeit b

10000000 loops, best of 3: 28.3 ns per loop


- 두번째(b) 방식의 코드가 더 간결(키 이름을 따옴표로 묶을 필요가 없다)
- 하지만 두 코드를 놓고 성능을 비교해 보면, dict()를 사용한 것이 3 배 정도 느리다!
- 그렇다면 이제 dict()를 사용한 모든 부분을 찾아서 첫번째(a) 방식으로 모두 변환하는 것이 좋을까?
- 영리한 프로그래머라면 이 부분을 수정하지 않고 그냥 둘 것
- 그보다는 **내부 순환문**과 같이 **실행 속도에 직접적으로 영향을 주는 곳을 찾아 최적화**하는 것이 좋음
- 그 이외의 장소에 대해서는 이 속도 차이가 거의 아무런 차이도 만들지 못함
- 만약 작성중인 프로그램의 성능이 이번 레시피에서 다룬 최적화로는 어림없을 만큼 중요하다면 **just-in-time(JIT) 컴파일 기술에 기반한 도구 사용**을 고려하도록 하자
- 예를 들어 PyPy 프로젝트는 파이썬 인터프리터의 대안 구현으로, 프로그램 실행을 분석하고 빈번히 실행되는 부분에 대해서 네이티브 코드를 생성함
- 이 기술을 사용하면 파이썬 프로그램의 실행 속도가 비약적으로 증가하고, 때로는 C로 작성한 프로그램 속도에 근접하거나 심지어 더 빨라지기도 함
- 하지만 안타깝게도 이 책을 집필하고 있는 현재 PyPy는 파이썬3를 완벽히 지원하지 않는다.
- Numba는 다이내믹 컴파일로서 데코레이터로 최적화하고 싶은 파이썬 함수에 주석을 단다.
- 이런 함수는 LLVM을 사용해서 네이티브 코드로 컴파일 됨
- 이 역시 엄청난 성능 향상을 가져오지만, PyPy와 마찬가지로 파이썬3 지원에 대해서는 아직 미비한 점이 존재
- 마지막으로 존 아우스터하우트(John Ousterhout)의 **"완벽한 성능 향상은 동작하지 않는 것을 동작 상태로 변이하는 것이다"**라는 말을 꼭 기억하자
- 꼭 필요하지 않다면 최적화에 대해서 걱정하지 않아도 좋다.
- 프로그램이 **올바르게 동작**하도록 만드는 것이 빠르게 실행하도록 만드는 것보다 **훨씬 중요**하다(**적어도 초기에는**)